In [11]:
import requests
import requests_cache
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
import time
import json
import re
import numpy as np
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [12]:
sleep_time = 1 # Đơn vị: giây 

In [13]:
ignoreRes = lambda s: '"incomplete_results":true' not in s.text
requests_cache.install_cache(expire_after=-1, filter_fn=ignoreRes)

In [2]:
def get_user_track(user_id_file, client_id):
    
    list_user_id = np.loadtxt(user_id_file, dtype=int) # User id list
    
    # Initialize playlist dataframe
    track = pd.DataFrame(columns = ['id', 'title', 'duration','full_duration', 'playback_count', 'likes_count', 'reposts_count',
                                   'comment_count', 'download_count', 'downloadable', 'commentable', 'has_downloads_left', 'state', 
                                    'genre', 'streamable', 'purchase_title', 'purchase_url', 'tag_list', 'visuals', 'user_id',
                                    'release_date', 'created_at', 'last_modified', 'license', 'policy', 'artwork_url', 'permalink_url'])
    
    # Attributes in the playlist dataframe (except tracks)
    attributes = ['id', 'title', 'duration','full_duration', 'playback_count', 'likes_count', 'reposts_count',
                                   'comment_count', 'download_count', 'downloadable', 'commentable', 'has_downloads_left', 'state', 
                                    'genre', 'streamable', 'purchase_title', 'purchase_url', 'tag_list', 'visuals', 'user_id',
                                    'release_date', 'created_at', 'last_modified', 'license', 'policy', 'artwork_url', 'permalink_url']
    
    # Cấu hình retry của request nếu gặp lỗi
    retries = Retry(total=6, backoff_factor=sleep_time, status_forcelist=[403, 500, 502, 503, 504 ])
    s = requests.Session()
    s.mount('https://', HTTPAdapter(max_retries=retries))

    for user_id in list_user_id:    
        checkpoint = len(track)
        
        # Get request with corresponding user_id
        url = f'https://api-v2.soundcloud.com/users/{user_id}/tracks?client_id={client_id}&limit=100'
        Print('User id:', user_id)
        print(' Get:', url)
        print('  Status: Doing')
        
        # Kiem duyet user nay da xong chua
        this_user_done = False
        while not this_user_done:
            try:
                re = s.get(url)
            except Exception as e:
                f e != KeyboardInterrupt:
                    print("  Error or timeout! Status code:",re.status_code)
                    print("  Ignore this playlist")
                    continue
                else:
                    return playlist

            # Loop for checking if can make a request
            canGet = False
            while not canGet:
                # Sleep then make the request again
                if not re.ok:
                    time.sleep(sleep_time)
                    re = requests.get(url)
                    print('  Status: Not ok')
                    continue

                # Check request timeout
                # If it is timeout, sleep then make the request again
                if re.status_code == 504:
                    print('  Status: error', re.status_code)
                    time.sleep(sleep_time)
                    re = requests.get(url)
                    continue

                if re.status_code == 429:
                    print('  Status: Exceeds an API rate limit')
                    return track

                # Convert Json string into python dictionary
                json_pydata = json.loads(re.text)

                # Request is done
                canGet = True

            print('  Status: Done')

            # Loop each playlist of the user
            for tr in json_pydata['collection']:

                temp = {} # Temporary dictionary for storing the current playlist record

                # Get corresponding attributes into temp
                for key in attributes:
                    temp[key] = tr[key]

                # Append the record into the dataframe
                track = track.append(temp, ignore_index=True)
                
            if json_pydata['next_href']:
                if len(track) - checkpoint >= 21892:
                    this_user_done = True
                    print(' # current tracks:', len(track))
                    continue
                
                next_url = json_pydata['next_href']
                url = next_url[:next_url.find('offset')] + f'client_id={client_id}&' + next_url[next_url.find('offset'):]
                print(' Get:', url)
                print('  Status: Doing')
            else:
                this_user_done = True
                print(' # current tracks:', len(track))
        
    return track

In [22]:
track = get_user_track('user_id.txt', '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn')

Get: https://api-v2.soundcloud.com/users/630253005/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/217441590/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/324531068/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/525378972/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/9855085/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/732907996/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/311417284/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/331219188/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Statu

Status: Done
Get: https://api-v2.soundcloud.com/users/54479107/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/607205031/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/819670534/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/745766704/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/68715998/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/18594680/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/374261675/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/17603266/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4

Status: Done
Get: https://api-v2.soundcloud.com/users/14520673/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/177736945/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/19252023/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/68515/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/48421686/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/830729737/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/18780390/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/828682924/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQS

Status: Done
Get: https://api-v2.soundcloud.com/users/808398/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/143717076/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/959200633/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/474568065/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/97125681/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/407692290/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/894482785/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/231916775/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4

Status: Done
Get: https://api-v2.soundcloud.com/users/346386776/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/318348605/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/114362015/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/53811920/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/3971992/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/90730831/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/117583159/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/759131683/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4

...

Status: Done
Get: https://api-v2.soundcloud.com/users/567991422/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/138919568/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/1017358915/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/249653632/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/182013524/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/52019567/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/20772533/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/893883/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4

Status: Done
Get: https://api-v2.soundcloud.com/users/40159516/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/131436871/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/124103716/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/385729871/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/78707508/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/241539717/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/956881543/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/103235250/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaV

Status: Done
Get: https://api-v2.soundcloud.com/users/574896/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/876356704/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/312392578/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/131440780/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/466290561/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/142328315/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/148737351/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/537292359/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm

Status: Done
Get: https://api-v2.soundcloud.com/users/2492807/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/18825821/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/940944472/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/6306315/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/29989882/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/624524112/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/7533219/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/112554881/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQS

Status: Done
Get: https://api-v2.soundcloud.com/users/205015297/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/126176881/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/32503304/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/8139697/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/41691970/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/205000014/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/569972175/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Get: https://api-v2.soundcloud.com/users/302229472/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4

In [25]:
track.to_csv('track.csv')

In [26]:
df = pd.read_csv('track.csv')
df.head()

,Unnamed: 0,id,title,duration,full_duration,likes_count,reposts_count,comment_count,download_count,downloadable,genre,created_at
0,0,1142178148,Daydream【Free Download】,214779,214779,80.0,47,0.0,0.0,False,Dance & EDM,2021-10-15T09:37:00Z
1,1,1130272750,Cozy Fall【Free Download】,186253,186253,113.0,79,2.0,0.0,False,Chill Out,2021-09-24T08:55:16Z
2,2,1126200826,No Regrets【Free Download】,280633,280633,337.0,284,179.0,0.0,False,Dance & EDM,2021-09-17T07:35:14Z
3,3,1118068150,Endless Summer【Free Download】,249704,249704,333.0,292,192.0,0.0,False,Dance & EDM,2021-09-03T09:40:32Z
4,4,1109341801,Fly High【Free Download】,217887,217887,95.0,68,0.0,0.0,False,Indie,2021-08-20T05:24:49Z
